In [1]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install statsmodels
%pip install pandas_datareader
%pip install datetime
%pip install yfinance
%pip install sklearn
%pip install PyPortfolioOpt


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 3.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 2.4 MB/s eta 0:00:004.6 MB/s eta 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 1.4 MB/s eta 0:

### Load SP500 stocks prices data.

In [7]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
import os
import certifi

os.environ['SSL_CERT_FILE'] = certifi.where()

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-05-31'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-06-02 A        43.071747   45.900002   46.150002   45.820000   45.910000   
           AAL      30.435610   31.670000   32.070000   31.580000   31.670000   
           AAPL     22.438629   24.430000   24.459999   24.157499   24.400000   
           ABBV     46.128258   65.089996   65.150002   62.830002   62.869999   
           ABT      34.118515   39.529999   39.639999   39.349998   39.500000   
...                       ...         ...         ...         ...         ...   
2024-05-30 XYL     139.399994  139.399994  140.160004  137.059998  138.050003   
           YUM     134.839996  134.839996  135.490005  134.360001  135.059998   
           ZBH     114.440002  114.440002  116.220001  114.370003  115.459999   
           ZBRA    316.619995  316.619995  319.549988  315.309998  319.549988   
           ZTS     169.440002  169.440002  174.169998  168.309998  170.679993   

Price                   volume  
date       ticker               
2016-06-02 A         2209400.0  
           AAL       6813300.0  
           AAPL    160766400.0  
           ABBV     11628000.0  
           ABT      12758500.0  
...                        ...  
2024-05-30 XYL       1751500.0  
           YUM       1587600.0  
           ZBH       1731100.0  
           ZBRA       233800.0  
           ZTS       2904700.0  

[992563 rows x 6 columns]

### Calculate features and technical indicators for each stock.
- Garman-Klass Volatility: A volatility measure that considers the high, low, opening, and closing prices to estimate the price variability. It is considered more accurate than traditional measures like standard deviation

- RSI (Relative Strength Index): A momentum oscillator that measures the speed and change of price movements, usually over a 14-period interval. An RSI above 70 indicates overbought conditions, while an RSI below 30 indicates oversold conditions.

- Bollinger Bands: Consist of three lines – the middle band is a simple moving average, and the upper and lower bands are standard deviations away from the middle band. They provide a relative definition of high and low prices.

- ATR (Average True Range): Measures market volatility by decomposing the entire range of an asset price for a given period. It is used to identify potential breakouts and volatility.

- MACD (Moving Average Convergence Divergence): A trend-following momentum indicator that shows the relationship between two moving averages of a security’s price. It helps identify potential buy and sell signals.

- Dollar Volume: Represents the total traded value of a stock, calculated by multiplying the adjusted closing price by the volume and converting it into millions. It helps identify stocks with high trading activity.

In [8]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-06-02 A        43.071747   45.900002   46.150002   45.820000   45.910000   
           AAL      30.435610   31.670000   32.070000   31.580000   31.670000   
           AAPL     22.438629   24.430000   24.459999   24.157499   24.400000   
           ABBV     46.128258   65.089996   65.150002   62.830002   62.869999   
           ABT      34.118515   39.529999   39.639999   39.349998   39.500000   
...                       ...         ...         ...         ...         ...   
2024-05-30 XYL     139.399994  139.399994  140.160004  137.059998  138.050003   
           YUM     134.839996  134.839996  135.490005  134.360001  135.059998   
           ZBH     114.440002  114.440002  116.220001  114.370003  115.459999   
           ZBRA    316.619995  316.619995  319.549988  315.309998  319.549988   
           ZTS     169.440002  169.440002  174.169998  168.309998  170.679993   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2016-06-02 A         2209400.0         -0.001547        NaN       NaN   
           AAL       6813300.0         -0.000492        NaN       NaN   
           AAPL    160766400.0         -0.002635        NaN       NaN   
           ABBV     11628000.0         -0.036380        NaN       NaN   
           ABT      12758500.0         -0.008259        NaN       NaN   
...                        ...               ...        ...       ...   
2024-05-30 XYL       1751500.0          0.000214  55.550949  4.919935   
           YUM       1587600.0          0.000034  44.662510  4.897556   
           ZBH       1731100.0          0.000098  34.666275  4.750956   
           ZBRA       233800.0          0.000056  55.681767  5.740694   
           ZTS       2904700.0          0.000565  52.105516  5.109968   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2016-06-02 A            NaN       NaN       NaN       NaN      95.162717  
           AAL          NaN       NaN       NaN       NaN     207.366940  
           AAPL         NaN       NaN       NaN       NaN    3607.377629  
           ABBV         NaN       NaN       NaN       NaN     536.379381  
           ABT          NaN       NaN       NaN       NaN     435.301074  
...                     ...       ...       ...       ...            ...  
2024-05-30 XYL     4.958433  4.996931  0.642582  1.135583     244.159089  
           YUM     4.926031  4.954507  0.309709 -0.495136     214.071978  
           ZBH     4.788652  4.826347 -0.749859 -0.802326     198.107088  
           ZBRA    5.769247  5.797800 -0.209171  0.676880      74.025755  
           ZTS     5.141643  5.173318  0.703673  0.711357     492.172375  

[992563 rows x 14 columns]

## Aggregate to monthly level and filter top 150 most liquid stocks for each month.
- To reduce training time and experiment with features and strategies, convert the business-daily data to month-end frequency.

In [9]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2016-07-31 A           75.466176   45.264515         -0.000743  63.936258   
           AAL        401.105355   34.116329         -0.000845  59.692644   
           AAPL      3118.136318   23.928873         -0.002745  67.567947   
           ABBV       350.139819   47.348209         -0.039624  65.472557   
           ABT        403.487977   38.861820         -0.006625  76.719032   
...                          ...         ...               ...        ...   
2024-05-31 XYL        201.683559  139.399994          0.000214  55.550949   
           YUM        264.300751  134.839996          0.000034  44.662510   
           ZBH        161.764632  114.440002          0.000098  34.666275   
           ZBRA        95.590943  316.619995          0.000056  55.681767   
           ZTS        444.613940  169.440002          0.000565  52.105516   

                     bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                    
2016-07-31 A       3.746726  3.795039  3.843352 -1.321277  0.095989  
           AAL     3.334677  3.509816  3.684955  0.652664  1.810152  
           AAPL    3.110997  3.162217  3.213437 -1.217319 -0.112978  
           ABBV    3.809448  3.841618  3.873788 -1.328258  0.179465  
           ABT     3.562258  3.628716  3.695173 -1.215201  0.815595  
...                     ...       ...       ...       ...       ...  
2024-05-31 XYL     4.919935  4.958433  4.996931  0.642582  1.135583  
           YUM     4.897556  4.926031  4.954507  0.309709 -0.495136  
           ZBH     4.750956  4.788652  4.826347 -0.749859 -0.802326  
           ZBRA    5.740694  5.769247  5.797800 -0.209171  0.676880  
           ZTS     5.109968  5.141643  5.173318  0.703673  0.711357  

[46861 rows x 9 columns]

- Calculate 5-year rolling average of dollar volume for each stocks before filtering.

In [10]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2017-06-30 AAL      48.838982         -0.000020  59.919035  3.858655   
           AAPL     33.703403         -0.001808  43.606678  3.515703   
           ABBV     53.369370         -0.036447  71.506040  3.924985   
           ABT      43.018219         -0.006063  67.057310  3.734658   
           ACN     111.341927         -0.003969  49.971662  4.703720   
...                       ...               ...        ...       ...   
2024-05-31 VZ       40.330002          0.000139  53.645621  3.682184   
           WFC      59.209999          0.000073  49.647755  4.087612   
           WMT      64.889999          0.000019  67.387915  4.072496   
           XOM     113.989998          0.000038  45.434773  4.732214   
           ZTS     169.440002          0.000565  52.105516  5.109968   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2017-06-30 AAL     3.891033  3.923412  0.508692  0.706175  
           AAPL    3.567603  3.619503 -1.085334 -0.452409  
           ABBV    3.972575  4.020166 -1.528941  0.404157  
           ABT     3.772220  3.809783 -1.332634  0.466136  
           ACN     4.735047  4.766373 -1.096257 -0.200666  
...                     ...       ...       ...       ...  
2024-05-31 VZ      3.707925  3.733666 -0.695211 -0.141973  
           WFC     4.119678  4.151744 -0.261138  0.109598  
           WMT     4.149212  4.225929  0.943250  2.638468  
           XOM     4.763441  4.794668  0.534310 -0.668915  
           ZTS     5.141643  5.173318  0.703673  0.711357  

[12516 rows x 8 columns]